<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/LazyPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lazypredict
!pip install optuna

In [ ]:
!sudo apt-get install build-essential swig
!pip install auto-sklearn==0.11.1

In [1]:
from autosklearn.regression import AutoSklearnRegressor

/usr/local/lib/python3.7/dist-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
import requests
from scipy import stats

import lightgbm as lgb

In [3]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

In [4]:
train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [5]:
train.shape, test.shape

((14687, 14), (14443, 14))

In [6]:
# train = pd.concat([train, target], axis=1)

In [6]:
features = [f for f in train.columns if 'Feature' in f]
len(features)

14

In [ ]:
# valid = train[train['moons'] > 34]
# valid = valid.reset_index(drop=True)

# train = train[train['moons'] <= 34]
# train.reset_index(drop=True, inplace=True)

# cols_drop = ['id', 'moons']
# train.drop(columns= cols_drop, inplace=True)
# valid.drop(columns= cols_drop, inplace=True)


In [7]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

In [58]:
def additional_features(df):
    df['Feature_4+Feature_14-Feature_3*Feature_7'] = df['Feature_4'] + df['Feature_14'] - df['Feature_3'] * df['Feature_7']
    df['Feature_4+Feature_14-Feature_3*Feature_5'] = df['Feature_4'] + df['Feature_14'] - df['Feature_3'] * df['Feature_5']
    df['Feature_4+Feature_14+Feature_2-Feature_5'] = df['Feature_4'] + df['Feature_14'] + df['Feature_2'] - df['Feature_5']
    df['Feature_4+Feature_14-Feature_1*Feature_3'] = df['Feature_4'] + df['Feature_14'] - df['Feature_1'] * df['Feature_3']
    df['Feature_4+Feature_14-Feature_11*Feature_3'] = df['Feature_4'] + df['Feature_14'] - df['Feature_11'] * df['Feature_3']
    df['Feature_4+Feature_14-Feature_2*Feature_3'] = df['Feature_4'] + df['Feature_14'] - df['Feature_2'] * df['Feature_3']
    df['Feature_4+Feature_14-Feature_13*Feature_10'] = df['Feature_4'] + df['Feature_14'] - df['Feature_13'] * df['Feature_10']
    df['Feature_4+Feature_14-Feature_13*Feature_11'] = df['Feature_4'] + df['Feature_14'] - df['Feature_13'] * df['Feature_11']
    df['Feature_4+Feature_14-Feature_10*Feature_7'] = df['Feature_4'] + df['Feature_14'] - df['Feature_10'] * df['Feature_7']
    df['Feature_4+Feature_14-Feature_10*Feature_1'] = df['Feature_4'] + df['Feature_14'] - df['Feature_10'] * df['Feature_1']
    df['Feature_4+Feature_14-Feature_10*Feature_2'] = df['Feature_4'] + df['Feature_14'] - df['Feature_10'] * df['Feature_2']
    df['Feature_4+Feature_14-Feature_12*Feature_10'] = df['Feature_4'] + df['Feature_14'] - df['Feature_12'] * df['Feature_10']
    df['Feature_4+Feature_14-Feature_10*Feature_5'] = df['Feature_4'] + df['Feature_14'] - df['Feature_10'] * df['Feature_5']
    return df

In [59]:
train = additional_features(train)

In [60]:
train

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_4+Feature_14-Feature_3*Feature_7,Feature_4+Feature_14-Feature_3*Feature_5,Feature_4+Feature_14+Feature_2-Feature_5,Feature_4+Feature_14-Feature_1*Feature_3,Feature_4+Feature_14-Feature_11*Feature_3,Feature_4+Feature_14-Feature_2*Feature_3,Feature_4+Feature_14-Feature_13*Feature_10,Feature_4+Feature_14-Feature_13*Feature_11,Feature_4+Feature_14-Feature_10*Feature_7,Feature_4+Feature_14-Feature_10*Feature_1,Feature_4+Feature_14-Feature_10*Feature_2,Feature_4+Feature_14-Feature_12*Feature_10,Feature_4+Feature_14-Feature_10*Feature_5
0,1.00,1.00,0.0,0.00,1.00,0.50,1.00,0.0,1.0,0.0,0.0,0.25,0.00,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25
1,1.00,1.00,0.0,0.00,0.50,0.50,0.50,0.0,1.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.50,0.50,0.0,0.00,0.25,0.75,0.00,0.0,1.0,0.0,0.0,0.00,0.25,0.25,0.25,0.25,0.50,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25
3,0.75,0.75,0.0,0.00,0.50,1.00,0.00,0.0,0.0,0.0,1.0,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.00,-0.50,0.00,0.00,0.00,0.00,0.00
4,1.00,1.00,0.0,0.00,1.00,1.00,0.25,0.0,0.0,0.0,1.0,0.25,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.25,0.50,0.50,0.50,0.50,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14682,0.00,0.00,1.0,0.75,0.25,0.75,0.00,1.0,0.0,0.0,0.0,0.00,0.75,0.75,1.50,1.25,1.25,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50
14683,0.25,0.25,1.0,0.75,0.25,0.75,0.25,1.0,0.0,0.0,0.0,0.00,0.75,0.75,1.25,1.25,1.50,1.25,1.50,1.25,1.50,1.50,1.50,1.50,1.50,1.50,1.50
14684,0.25,0.25,1.0,1.00,0.25,0.75,0.00,1.0,0.0,0.0,0.0,0.00,0.00,0.00,1.00,0.75,1.00,0.75,1.00,0.75,1.00,1.00,1.00,1.00,1.00,1.00,1.00
14685,0.00,0.00,1.0,0.75,0.00,0.75,0.00,1.0,0.0,0.0,0.0,0.00,0.75,0.75,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50,1.50


In [62]:
features = [ f for f in train.columns if "Feature" in f]
len(features)

27

# R

In [63]:
from lazypredict.Supervised import LazyRegressor

X_train, X_test, y_train, y_test = train_test_split(train[features], target['target_r'], test_size=0.20, shuffle=False, random_state=60)
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=scorer,random_state=60 )
models, predictions = reg.fit(X_train, X_test, y_train, y_test, )
models.sort_values(by='scorer', ascending=False)

100%|██████████| 38/38 [02:03<00:00,  3.24s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken,scorer
Model,,,,,
MLPRegressor,-0.22,-0.21,0.37,8.41,0.12
AdaBoostRegressor,-0.02,-0.01,0.34,0.19,0.12
RANSACRegressor,-1.42,-1.40,0.53,0.25,0.11
SVR,-0.55,-0.53,0.42,14.45,0.10
OrthogonalMatchingPursuit,-0.02,-0.01,0.34,0.04,0.10
OrthogonalMatchingPursuitCV,-0.01,-0.00,0.34,0.11,0.10
NuSVR,-0.30,-0.29,0.39,17.29,0.10
ElasticNetCV,-0.00,0.01,0.34,0.76,0.10
LassoLarsCV,-0.00,0.01,0.34,0.22,0.10


In [66]:
from sklearn.svm import LinearSVR, NuSVR
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostRegressor

In [67]:
sc = StandardScaler()

In [69]:
X_train, X_test, y_train, y_test = train_test_split(train[features], target['target_r'], test_size=0.20, shuffle=False, random_state=60)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
model_r = AdaBoostRegressor(random_state=60)
model_r.fit(X_train, y_train)
preds = model_r.predict(X_test)
scorer(preds, y_test)

0.11656987920431583

# G

In [64]:
from lazypredict.Supervised import LazyRegressor
X_train, X_test, y_train, y_test = train_test_split(train[features], target['target_g'], test_size=0.20, shuffle=False, random_state=60)
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=scorer, random_state=60)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models.sort_values(by='scorer', ascending=False)

100%|██████████| 38/38 [01:57<00:00,  3.09s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken,scorer
Model,,,,,
LarsCV,-0.00,0.01,0.36,0.21,0.17
LassoLarsCV,-0.00,0.00,0.36,0.22,0.17
ElasticNetCV,-0.01,0.00,0.36,0.62,0.16
LassoCV,-0.01,0.00,0.36,0.59,0.16
OrthogonalMatchingPursuitCV,-0.02,-0.01,0.36,0.09,0.15
OrthogonalMatchingPursuit,-0.02,-0.01,0.36,0.04,0.15
XGBRegressor,-0.29,-0.28,0.40,1.01,0.13
BayesianRidge,-0.02,-0.01,0.36,0.05,0.11
NuSVR,-0.18,-0.17,0.39,17.44,0.11


In [72]:
from sklearn.linear_model import Lars, LarsCV
sc = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(train[features], target['target_g'], test_size=0.20, shuffle=False, random_state=60)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
model_g = LarsCV()
model_g.fit(X_train, y_train)
preds = model_g.predict(X_test)
scorer(preds, y_test)

0.1700573898411129

# B

In [65]:
from lazypredict.Supervised import LazyRegressor
X_train, X_test, y_train, y_test = train_test_split(train[features], target['target_b'], test_size=0.25, shuffle=False, random_state=60)
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=scorer, random_state=60)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models.sort_values(by='scorer', ascending=False)

100%|██████████| 38/38 [01:37<00:00,  2.56s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken,scorer
Model,,,,,
AdaBoostRegressor,0.04,0.05,0.33,0.21,0.27
LassoLarsCV,0.00,0.01,0.34,0.27,0.21
LassoCV,-0.01,0.00,0.34,0.61,0.21
ElasticNetCV,-0.01,0.00,0.34,0.64,0.21
OrthogonalMatchingPursuitCV,0.00,0.01,0.34,0.11,0.21
OrthogonalMatchingPursuit,0.00,0.01,0.34,0.04,0.21
LGBMRegressor,-0.11,-0.10,0.36,0.28,0.20
LarsCV,0.00,0.01,0.34,0.20,0.20
SGDRegressor,-0.07,-0.06,0.35,0.06,0.16


In [80]:
# model_dict = reg.provide_models(X_train, X_test, y_train, y_test)

In [73]:
from sklearn.ensemble import AdaBoostRegressor

In [74]:
sc = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(train[features], target['target_b'], test_size=0.25, shuffle=False, random_state=60)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
model_b = AdaBoostRegressor(random_state=60)
model_b.fit(X_train, y_train)
preds = model_b.predict(X_test)
scorer(preds, y_test)

0.27069109815038855

In [75]:
preds

array([0.50446979, 0.50446979, 0.50446979, ..., 0.50446979, 0.43159204,
       0.58071043])

In [76]:
test = additional_features(test)

In [77]:
test = sc.transform(test)

In [84]:
test.shape

(14443, 27)

In [78]:
prediction = pd.DataFrame()
prediction['target_r'] = model_r.predict(test)
prediction['target_g'] = model_g.predict(test)
prediction['target_b'] = model_b.predict(test)
prediction

,target_r,target_g,target_b
0,0.82,0.57,0.58
1,0.56,0.53,0.53
2,0.42,0.41,0.45
3,0.56,0.46,0.50
4,0.56,0.52,0.50
...,...,...,...
14438,0.42,0.34,0.45
14439,0.56,0.54,0.53
14440,0.56,0.48,0.45
14441,0.56,0.50,0.50


In [80]:
prediction.describe()

,target_r,target_g,target_b
count,14443.00,14443.00,14443.00
mean,0.56,0.49,0.50
std,0.12,0.07,0.08
min,0.31,0.30,0.30
25%,0.54,0.45,0.45
50%,0.56,0.50,0.50
75%,0.56,0.55,0.53
max,1.00,0.69,0.72


In [81]:
prediction[prediction.target_r < 0] = 0
prediction[prediction.target_g < 0] = 0
prediction[prediction.target_b < 0] = 0

prediction[prediction.target_r > 1] = 1
prediction[prediction.target_g > 1] = 1
prediction[prediction.target_b > 1] = 1


In [82]:
prediction.describe()

,target_r,target_g,target_b
count,14443.00,14443.00,14443.00
mean,0.56,0.49,0.50
std,0.12,0.07,0.08
min,0.31,0.30,0.30
25%,0.54,0.45,0.45
50%,0.56,0.50,0.50
75%,0.56,0.55,0.53
max,1.00,0.69,0.72


In [83]:
API_KEY = "0GIskIC9Rd1I8sMw6nStutbIbMx7AaZzAyNfuMOVAVWgJtNa1IVQHZAD3WPj" # <- HERE

r = requests.post("https://tournament.datacrunch.com/api/submission",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same as the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted :)
